 ###                 MULTIVARIATE LINEAR REGRESSION [OOP Edition]

@# nope!11!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!This notebook is divided into two parts. "Model 1" is my rendition of the MLR assignment.
"Model 2" is the OOP version of the former.
Both endeavour to predict employee salaries from different employee characteristics (or features).

In [ ]:
#Required Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from sklearn.linear_model import LinearRegression
sns.set()
from scipy import stats
import statsmodels.api as sm
# from statsmodels.formula.api import ols
import statsmodels
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings("ignore")

#### **This assignment addresses the following:**

[OOP for Data Science](#oop)
<a href='oop'> </a>

[Exploratory Data Analysis -EDA](#EDA) 
<a href='EDA'> </a>

[Feature Engineering](#feng)
<a href='feng'> </a>

[Correlation and Statistical Significance Analysis](#corr)
<a href='corr'> </a>

[~~Feature Selection~~](#select)
<a href='select'> </a>

[Model Training](#train)
<a href='train'> </a>

[Predictions](#predict)
<a href='predict'> </a>

[Model Evaluation](#eval)
<a href='eval'> </a>


### Navigation:

## [MODEL 1](#MLR)   



## [MODEL 2](#OOP)

In [ ]:
from src.Plotter import Histogram_Plotter, Scatter_Plot c

### Use the model you built in the MLR project to predict log-transformed salary (log_salary). Also create a second multiple regression model which does not include yearsrank as a feature. Save these model instances as model1 and model2. Remember to scale (standardise) the features before modelling. m

In [ ]:
oop = pd.read_csv('data/salary.csv')
oop = oop.drop("yearsrank", axis=1)

#Method to fix missing value
oop['salary'].fillna(oop['salary'].mean(),inplace = True)

certain markdowns have been omitted in model 1 because they would be repetitive from model 2

oop['log_salary'] = np.log(oop.salary)
Logarithmic transformations are used to make  better predicted outcomes from a linear reg model.

dummies = pd.get_dummies(oop['Field'])

dum = dummies.rename(columns={1:'engineering',2:'finance',3:'HR',4:'marketing'})

oop2 = pd.concat([oop, dum] ,axis=1, ignore_index=False)
oop2.drop("Field", inplace=True, axis=1)

### Train Test Split for Model 2 

#split data
x2 = oop.drop(['salary', 'log_salary'], axis=1)
y2 = oop["log_salary"]

x2 = sm.add_constant(x2)

X_train2,X_test2,Y_train2,Y_test2 = train_test_split(x2,y2,test_size=0.3, random_state=50)

model2 = sm.OLS(Y_train2, X_train2).fit()

model2.summary()

### MSE
errors_m1 = ErrorCalculator(Y_test, model1.predict(sm.add_constant(X_test)))
m1_mse = errors_m1.get_mse()
errors_m2 = ErrorCalculator(Y_test2, model2.predict(sm.add_constant(X_test2)))
m2_mse = errors_m2.get_mse()

print(f'Model1 MSE: {m1_mse}')
print(f'Model2 MSE: {m2_mse}')
print(f'MSE diff: {m2_mse - m1_mse}')

### RMSE

rmse_m1 = errors_m1.get_rmse()
rmse_m2 = errors_m2.get_rmse()

### RMSE

In [ ]:
print(f'Model1 RMSE: {rmse_m1}')
print(f'Model2 RMSE: {rmse_m2}')
print(f'MSE diff: {rmse_m2 - rmse_m1}')

Model2 has a higher RMSE. This can be attributed to a different feature being used.

### Plots

In [ ]:
# from Histogram_Plotter import Histogram_Plotter

In [ ]:
model1_hist = Histogram_Plotter(Y_test, model1.predict(sm.add_constant(X_test)))
model1_scatter = Scatter_Plot(Y_test, model1.predict(sm.add_constant(X_test)))

# Mode1 Histogram Plot
model1_hist.plot()

#mode2l1 Scatter Plot
model1_scatter.plot()

In [ ]:
model2_hist = Histogram_Plotter(Y_test2, model2.predict(sm.add_constant(X_test2)))
model2_scatter = Scatter_Plot(Y_test2, model2.predict(sm.add_constant(X_test2)))

# Mode1 Histogram Plot
model2_hist.plot()

#mode2l1 Scatter Plot
model1_scatter.plot()


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
class Plotter():
    def __init__(self, y, y_pred):
        self.y = y
        self.y_pred = y_pred
        
    def run_calcs(self):
        return self.y - self.y_pred
    
    def plot(self):
        f, axes = plt.subplots(1, 2,figsize=(15, 5))
        grid = plt.GridSpec(1, 2, wspace=0.5, hspace=0.3)
        plt.subplot(grid[0, 0])
        plt.hist(self.y - self.y_pred)
        plt.xlabel('residuals')
        plt.ylabel('frequency')
        return plt.show()

class Histogram_Plotter(Plotter):
    def __init__(self, y, y_pred):
        Plotter.__init__(self, y, y_pred)

class Scatter_Plot(Plotter):
    def __init__(self, y, y_pred):
        Plotter.__init__(self, y, y_pred)

    def plot(self):
        df = pd.DataFrame({"Actual": self.y, 
                            "Predicted": self.y_pred})
        df.plot.scatter(x="Actual", y="Predicted")
        plt.title("Predicted vs Actual Values")
        plt.xlabel("Actual")
        plt.ylabel("Prediction")
        return plt.show()

## MLR

<a id='EDA'></a> <div class="alert alert-block alert-info">

#### Exploratory Data Analysis (EDA)
- Data Ingestion
- Data Preprocessing
 </div>

In [ ]:
#Load Data
sal = pd.read_csv('data/salary.csv')

In [ ]:
#Examine DataSet
sal.info()

* Our target variable (salary) has one missing value.

In [ ]:
#Method to fix missing value
sal['salary'].fillna(sal['salary'].mean(),inplace = True)
#sal = sal.drop_duplicates()

* Missing value has been filled w/ mean. Duplicates would be dropped if dataset had any 
* Filled w/ mean instead of dropping because salary is the target variable and don't want to miss any insights it might bring even when it's just one value.

In [ ]:
sal.describe()

* The above table tells us the number of observations (514). We expect salary to have 514 because the missing value is now represented by the mean.
* The descriptive statistics table also tells us about the means, standard deviations, min and max values as well as the percentiles.


 <a id='feng'></a> <div class="alert alert-block alert-info">

#### Feature Engineering
 </div>

In [ ]:
import plotly.graph_objs as go

field = ['Engineering', 'Finance', 'Human Resources', 'Marketing']
trace = go.Pie(labels = field, values = sal.Field)
data = [trace]
layout = go.Layout(
   {"title":"Career Fields"})
fig = go.Figure(data,layout)
fig.show()

The purpose of this graph is to have an understanding of which career fields are the most dominant within the dataset. The Marketing profession accounts for 33.3%, Eng & HR are tied at second place leaving Finance in last.

In [ ]:
dummies = pd.get_dummies(sal['Field'])

dum = dummies.rename(columns={1:'engineering',2:'finance',3:'HR',4:'marketing'})

sal2 = pd.concat([sal, dum] ,axis=1, ignore_index=False)
sal2.drop("Field", inplace=True, axis=1)

In [ ]:
sal2

 <a id='corr'></a> <div class="alert alert-block alert-info">

#### Correlation Analysis and Statistical Significance 
 </div>

> __Model 1 Features:__  exprior, yearsworked, yearsrank, market, degree, otherqual, position, male,
Field_dummyvariable1, Field_dummyvariable2, Field_dummyvariable3, yearsabs

In [ ]:
sal_corr = sal2.corr()
sal_corr.style.background_gradient(cmap = 'coolwarm')

__Everything has a correlation but it is the strength of the correlation that we are interested in.__
* Based on the above table these features are good predictors for salary; yearsworked, yearsrank, position and Field

* As per assignment instructions ALL features will be used for Model 1.


<a id='select'></a> <div class="alert alert-block alert-info">
    
      N/A FOR THIS ASSIGNMENT!  

# ~~Feature Selection~~

 </div>

 <a id='train'></a> <div class="alert alert-block alert-info">

#### Model Training
 </div>

__Dataset is split into a train & test set. 
70% of the data will go into the training set and the remaining 30% will be used for testing.__

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
sal2['log_salary'] = np.log(sal2.salary)

In [ ]:
#split data
x = sal2.drop(['salary','log_salary'] , axis=1)
y = sal2["log_salary"]

x = sm.add_constant(x)

X_train,X_test,Y_train,Y_test = train_test_split(x,y,test_size=0.3, random_state=50)

In [ ]:
model1 = sm.OLS(Y_train, X_train).fit()

In [ ]:
model1.summary()

R-squared value reveals the quality of the regression model. It describes the relationships between dependent and independent variables in a model. The R-squared value for this model was 0.66. Now, having used more features & did log transformation, it is now 0.83.
This means the accuracy of the model is approx 83%.

<a id='predict'></a> <div class="alert alert-block alert-info">

#### Predictions & Model Testing


Predicting salary using the test set.
 </div>

In [ ]:
predict_salary = model1.predict(X_test)

We use the __X_test__ data to pass in features the model has never seen

In [ ]:
#dataframe for predicted values
df = pd.DataFrame(predict_salary)
df.rename(columns={0:'predicted_salary'},inplace=True)
# df = df.astype(int)
df.head(2)

In [ ]:
df_compare = pd.concat([Y_test,df],axis=1)

In [ ]:
df_compare

### Analyse Distribution of Residuals

In [ ]:
sns.distplot((Y_test - predict_salary))
plt.title("Distribution of Residuals")
plt.show()

- The figure is normally distributed
- This is a good sign because it means this model is a correct choice for the data



<a id='eval'></a> <div class="alert alert-block alert-info">

#### Model Evaluations
Regression Evaluation Metrics

 </div>

#### 3 common evaluation metrics for regression problems:

**Mean Absolute Error** (MAE) is the mean of the absolute value of the errors:

$$\frac 1n\sum_{i=1}^n|y_i-\hat{y}_i|$$

**Mean Squared Error** (MSE) is the mean of the squared errors:

$$\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2$$

**Root Mean Squared Error** (RMSE) is the square root of the mean of the squared errors:

$$\sqrt{\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2}$$

All of these are **loss functions**. We want to minimize them to create the best model.

In [ ]:
from sklearn import metrics

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
print('MSE:', metrics.mean_squared_error(Y_test, predict_salary))

In [ ]:
print('testRMSE:', np.sqrt(metrics.mean_squared_error(Y_test, predict_salary)))

In [ ]:
print('trainsetRMSE:', np.sqrt(metrics.mean_squared_error(Y_train, model1.predict(X_train))))

- RMSE indicates the absolute fit of the model to the data (how close the actual data points are to the model's predicted ones.)
- Test RMSE is bigger than the Trainset RMSE
- RMSE is a good measure of how accurately the model predicts the target variable. It is the most important criteria for fit if the main purpose of the model is prediction.

#### Mean Absolute Error(MAE) is one of the many metrics for summarizing and assessing the quality of a machine learning model.


In [ ]:
from sklearn.metrics import mean_absolute_error
print('MAE:', metrics.mean_absolute_error(Y_test, predict_salary))

In [ ]:
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
mape(Y_test, predict_salary)

- The mean absolute percentage error (MAPE) is a statistical measure of how accurate a forecast system is. It measures this accuracy as a percentage.
- (MAPE) works best if there are no extremes to the data
- Since MAPE is a measure of error, high numbers are bad and low numbers are good. Initially mape % was 12.2 but now it has reduced to 1.1% post-log transformation. 


In [ ]:
metrics.r2_score(Y_test, predict_salary)

### Prediction Error Plot

In [ ]:
# from sklearn.linear_model import Lasso
# from yellowbrick.regressor import PredictionError

In [ ]:
# mod = Lasso()
# visualizer = PredictionError(mod)

# #Fit training data to visualizer
# visualizer.fit(X_train, Y_train)

# #Evaluate model on the test data
# visualizer.score(X_test, Y_test)

# visualizer.show()                 

- A __Prediction Error Plot__ shows the actual vales from the dataset against the predicted values generated by the model.
- This allows us to see how much variance is in the model.
- The Line of Best Fit is used to express a relationship in a scatter plot of different data points.
- It is an output of regression analysis and can be used as a prediction tool for indicators.

In [ ]:
import statsmodels.api as sm

### Calculate the standardised residuals resid() & standardised predicted values fittedvalues()

In [ ]:
residuals  = model1.resid

fittedv   = model1.fittedvalues

#### Plot the residuals versus the predicted values using seaborn’s residplot with fitted values as the x parameter, and the dependent variable as y, specify lowess=True. 


In [ ]:
sns.residplot(fittedv,residuals, lowess=True, color='maroon')
plt.title('Residuals vs Predicted Values')
plt.ylabel('Fitted Values')
plt.xlabel('Residuals')
plt.show()

## OOP

<div class="alert alert-block alert-info">

#### OOP for Data Science
 </div>

### Assignment Outline

It is efficient to put machine learning models and other data science techniques into classes so that we can reuse them later and change attributes without changing the code behind these models. Independent concepts can also be put into independent classes: for example, the functioning of a cross-validate class should not affect the functioning of a linear regression class.

> Create a class called __ErrorCalculator__ that has methods to compute __the residuals, standardised residuals, Mean Squared Error (MSE) and Root Mean Squared Error (RMSE).__ 
Name these methods __get_residuals__, __get_standardised_residuals__, __get_mse__ and __get_rmse__ respectively. 
You can also have a method, __error_summary__ that prints the average, minimum and maximum of the standardised residuals, as well as the MSE and RMSE.
The class should have the following parameters:

      y: A 1D array of the target variable, size n_observations
      y_pred: A 1D array of the predicted values of the target variable, size n_observations

In [ ]:
from src.ErrorCalculator import ErrorCalculator

> Create a generic class called __Plotter__. This class should have a method, run_calculations, to calculate the residuals if they have not yet been calculated, and a method plot, which simply plots a histogram of the residuals.
As before, the class should have the following parameters:

     y: A 1D array of the target variable, size n_observations
     y_pred: A 1D array of the predicted values of the target variable, size n_observations

> Create two child classes, __HistogramPlotter__ and __ScatterPlotter__, that both inherit from Plotter. As the name suggests, HistogramPlotter.plot() should return a histogram of the residuals, whereas ScatterPlotter.plot() should return scatterplots of the residual versus predicted values and the predicted versus observed values. 